# Research Proposal: Deep Learning Mechanisms for Automated Customer Care Interviews Using Video Monitoring System Data
**Step 1: Face recognition**
* As the evaluation sheet consist of two type of evaluations ; Image and Audio . In this stage will start with the Images in order to compute the face scene time ,Is it sufficient to proceed with emotional elements ? otherwise will depend on Audio only .


# Define Libraries

In [1]:
import face_recognition
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
#from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
#from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from datetime import datetime
import time
import os
import pickle
from pydub import AudioSegment
from pydub.utils import make_chunks
import ffmpy
from tqdm import tqdm
import glob
#import soundfile,librosa
#import librosa
from PIL import Image
from math import hypot

In [2]:
Videos_path="D:/VMS/Sample Videos/test999/"
output_path="D:/VMS/del/RAW/"

In [3]:
def mp4_2_img(videoFile,output_dir):
#path and directories preperation
    Folder_name=os.path.splitext(videoFile)[0]

    if not os.path.exists(output_path+Folder_name):
        os.makedirs(output_path+Folder_name)
        os.makedirs(output_path+Folder_name+"/"+Folder_name+"_Sound")
        os.makedirs(output_path+Folder_name+"/"+Folder_name+"_Image")

    wav_dir=output_path+Folder_name
    images_dir= output_path+Folder_name+"/"+Folder_name+"_Image"
    Sound_dir=output_path+Folder_name+"/"+Folder_name+"_Sound"

    videoFile=Videos_path+videoFile

    # Mp4 to JPG process
    count=0
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) # get the frame rate
    x=0
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0): # We are getting the last frame from each second .
            filename =images_dir+"\\"+Folder_name+"_frame_%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

    #Sound part

    # to carry naming and convert to wav in order to chunk them . 
    soundfilename =wav_dir+"\\"+Folder_name+"_sound.wav"
    ff = ffmpy.FFmpeg(
        inputs={videoFile: None},
        outputs={soundfilename: '-ac 1 -ar 16000 -f wav'}  #-ac 2 -f wav
    )
    ff.cmd
    ff.run()

    time.sleep(1) #little delay to ensure creating the file over the network

    myaudio = AudioSegment.from_file(soundfilename , "wav") 
    chunk_length_ms = 10000 # pydub calculates in millisec
    chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

    #Export all of the individual chunks as wav files

    for i, chunk in enumerate(chunks):
        chunk_name = Sound_dir+"\\"+Folder_name+"_chunk_%d.wav" % i

        chunk.export(chunk_name, format="wav")

#     if os.path.exists(soundfilename):
#         os.remove(soundfilename)
#     else:
#         print("The file does not exist")

    return 1

In [4]:
Videos_list=os.listdir(Videos_path)
print ("Total of Videos =" + str(len(Videos_list)) )
for vdo in Videos_list:  #loop all to Convert to JPG!
    if(mp4_2_img(vdo,output_path)):
        print(vdo+" has been Converted to JPG and WAV splitted Successfully ..")

Total of Videos =60
P06681_06_06_2019_081946_wc.mp4 has been Converted to JPG and WAV splitted Successfully ..
P06682_17_06_2019_024317_wc.mp4 has been Converted to JPG and WAV splitted Successfully ..
P06682_21_06_2019_064900_wc.mp4 has been Converted to JPG and WAV splitted Successfully ..
P08446_18_06_2019_213500_wc.mp4 has been Converted to JPG and WAV splitted Successfully ..
P08446_27_06_2019_151031_wc.mp4 has been Converted to JPG and WAV splitted Successfully ..
P08446_29_06_2019_205931_wc.mp4 has been Converted to JPG and WAV splitted Successfully ..
P08446_30_06_2019_213152_wc.mp4 has been Converted to JPG and WAV splitted Successfully ..
P08449_27_06_2019_195811_wc.mp4 has been Converted to JPG and WAV splitted Successfully ..
P08449_29_06_2019_184520_wc.mp4 has been Converted to JPG and WAV splitted Successfully ..
P08453_15_06_2019_184841_wc.mp4 has been Converted to JPG and WAV splitted Successfully ..
P08453_25_06_2019_122023_wc.mp4 has been Converted to JPG and WAV spli

In [5]:
# img = plt.imread('d:/VMS/del/P08446_30_06_2019_213152_wc/P08446_30_06_2019_213152_wc_Image/P08446_30_06_2019_213152_wc_frame_5.jpg')   # reading image using its name
# plt.imshow(img)

In [6]:
Staff_df = pd.read_excel("D:/VMS/del/VMS_Staff.xlsx",header=0)
Staff_df["AgentID"]= Staff_df["AgentID"].astype(str)

In [ ]:
Staff_df.iloc[0:5]


In [8]:
def staff_Is(fld_name):
    filt=(Staff_df['file_name']==fld_name)
    stf_Is=Staff_df.loc[filt,'AgentID'].item()
    return stf_Is

In [9]:

staff_images_path='D:/VMS/STAFF PIC'
images=[]
StaffNumbers=[]
staff_images_list=os.listdir(staff_images_path)
#print(staff_images_list)
for clss in staff_images_list:
    curImg = cv2.imread(f'{staff_images_path}/{clss}')
    images.append(curImg)
    StaffNumbers.append(os.path.splitext(clss)[0])
#print(StaffNumbers)

In [10]:
def imgencoding(x):
    encodeList=[]
    for img in x:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #conv image to RGB (search about it)
        encode =face_recognition.face_encodings(img)
        if len(encode) >0:
            encode =face_recognition.face_encodings(img)[0]
            encodeList.append(encode)
            
    return encodeList

In [11]:
#****************This part to read and write to pickle file ** for update   
encodeStaffListKnown=imgencoding(images) #Uncomment if new image added .
file = open('D:/VMS/StaffListKnown.dat', 'wb') #safe them , cuz process took about 3 mins
pickle.dump(encodeStaffListKnown, file)
file.close()
#****************This part to read and write to pickle file ** for update   

In [12]:
#***************************** This part to read from pickle file .dat
#Load face encodings
# encodeStaffListKnown = pickle.load(open('D:/VMS/StaffListKnown.dat','rb'))
#***************************** end  part to read from pickle file .dat

In [13]:
print(len(encodeStaffListKnown)) #to print how many pic are there BUT takes time
print("Done..!")

33
Done..!


In [14]:
def SceneFun(WhoIs,sceneDF):
    if WhoIs not in sceneDF.values:
        #print('Element not exists in Dataframe')
        Pepole = {'StaffID':WhoIs,'FrameTimes':1}
        sceneDF=sceneDF.append(Pepole,ignore_index=True)
            
    else:
        #print('Element exists in Dataframe')
        filt=(sceneDF['StaffID']==WhoIs)
        sceneDF.loc[filt,'FrameTimes']=sceneDF.loc[filt,'FrameTimes']+1
    return(sceneDF)

In [15]:
#Will add eye later
def get_landmarks(face_landmarks_list):
    landmarks_vectorised = []
    xlist = []
    ylist = []
    for face_landmarks in face_landmarks_list:

        # Print the location of each facial feature in this image
        for facial_feature in face_landmarks.keys():
            print(".".format(facial_feature, face_landmarks[facial_feature]),end ='')

        # Let's trace out each facial feature in the image with a line!
        for facial_feature in face_landmarks.keys():
            # d.line(face_landmarks[facial_feature], width=5)
            # print(face_landmarks[facial_feature])
            # print(face_landmarks[facial_feature][2][0])
            for sub_feature in face_landmarks[facial_feature]: #Store X and Y coordinates in two lists
                xlist.append(float(sub_feature[0]))
                ylist.append(float(sub_feature[1]))
                
        xlist.pop(71)
        ylist.pop(71)

        xlist.pop(67)
        ylist.pop(67)

        xlist.pop(66)
        ylist.pop(66)
        
        xlist.pop(60)
        ylist.pop(60)

    xmean = np.mean(xlist) #Find both coordinates of centre of gravity
    ymean = np.mean(ylist)
    xcentral = [(x-xmean) for x in xlist] #Calculate distance centre <-> other points in both axes
    ycentral = [(y-ymean) for y in ylist]

    for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
#         landmarks_vectorised.append(w)
#         landmarks_vectorised.append(z)
        meannp = np.asarray((ymean,xmean))
        coornp = np.asarray((z,w))
#         dist = np.linalg.norm(coornp-meannp)
        dist = np.linalg.norm(coornp-(112,112))
        landmarks_vectorised.append(dist)
#         landmarks_vectorised.append((math.atan2(y, x)*360)/(2*math.pi))

    return landmarks_vectorised

In [16]:
def get_gaze_ratio(eye_points,img4i):
    gray = cv2.cvtColor(img4i, cv2.COLOR_BGR2GRAY)
    left_eye_region = np.array([(eye_points[0][0],eye_points[0][1]),
                                (eye_points[1][0],eye_points[1][1]),
                                (eye_points[2][0],eye_points[2][1]),
                                (eye_points[3][0],eye_points[3][1]),
                                (eye_points[4][0],eye_points[4][1]),
                                (eye_points[5][0],eye_points[5][1])], np.int32)
    height, width, _ = img4i.shape
    mask = np.zeros((height, width), np.uint8)
    cv2.polylines(mask, [left_eye_region], True, 255, 2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)

    min_x = np.min(left_eye_region[:, 0])
    max_x = np.max(left_eye_region[:, 0])
    min_y = np.min(left_eye_region[:, 1])
    max_y = np.max(left_eye_region[:, 1])

    gray_eye = eye[min_y: max_y, min_x: max_x]
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    if len(gray_eye) == 0:
        return 0
    
    height, width = threshold_eye.shape
    left_side_threshold = threshold_eye[0: height, 0: int(width / 2)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0: height, int(width / 2): width]
    right_side_white = cv2.countNonZero(right_side_threshold)

    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else:
        gaze_ratio = left_side_white / right_side_white
    return gaze_ratio*100

In [17]:
def get_blinking_ratio(eye_points):
    left_point = eye_points[0]
    right_point = eye_points[3]
    center_top = midpoint2(eye_points[1], eye_points[2])
    center_bottom = midpoint2(eye_points[5], eye_points[4])

    #hor_line = cv2.line(frame, left_point, right_point, (0, 255, 0), 2)
    #ver_line = cv2.line(frame, center_top, center_bottom, (0, 255, 0), 2)

    hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    if ver_line_lenght  == 0:
        ratio = 0
    else:
        ratio = hor_line_lenght / ver_line_lenght
        
    return ratio*100

In [18]:
def midpoint2(p1 ,p2):
    return int((p1[0] + p2[0])/2), int((p1[1] + p2[1])/2)

In [19]:
#Extract features from faces
def extract_Face_LandMarks(F_image, img_path, F_faceLoc,eye_rgb):
    
    file_location = os.path.dirname(os.path.dirname(img_path)) 
    file_naming=os.path.basename(img_path)
    new_path_name=file_location+"/faces/"
    if not os.path.exists(new_path_name):
        os.makedirs(new_path_name)

    # if  staff_Is !=  F_Target_Staff and F_faceLoc !=0:
    #     return np.zeros(1)

    if F_faceLoc == 0 :
        os.remove(img_path)
        if os.path.exists(new_path_name+file_naming):
            os.remove(new_path_name+file_naming)
        return None
#         face_df=pd.DataFrame(0, index=range(420), columns = ['face_encodeing'])
#         xx=face_df.to_numpy(dtype='float32') #len = 128

    else:
        top=F_faceLoc[0]
        right=F_faceLoc[1]
        bottom=F_faceLoc[2]
        left=F_faceLoc[3]
        loc_face=[(top, right, bottom, left)]
        

        #face_landmark_list =face_recognition.face_landmarks(F_image,F_faceLoc)
        face_landmark_list=face_recognition.face_encodings(F_image,loc_face)
        face_landmark2 =face_recognition.face_landmarks(F_image,loc_face)
        eye=[]
        for face_landmarks in face_landmark2:
            for facial_feature in face_landmarks.keys():
                if facial_feature =="left_eye" or facial_feature =="right_eye":
                #print(facial_feature, face_landmarks[facial_feature])
                    eye.append(face_landmarks[facial_feature])
                
        blinking_ratio=get_blinking_ratio(eye[0])
        gaze_ratio_left_eye = get_gaze_ratio(eye[0],F_image)
        gaze_ratio_right_eye = get_gaze_ratio(eye[1],F_image)
        #gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye) / 2
        eye_info=[(blinking_ratio,gaze_ratio_left_eye,gaze_ratio_right_eye)]
        get_landmarks_all=get_landmarks(face_landmark2) #len = 68
        face_df =np.array(face_landmark_list[0],dtype='float32') #To add facedetails again 
        xx=np.array(get_landmarks_all,dtype='float32')
#         xx_Face=np.append(xx1,face_df)
#         xx=np.append(xx,eye_info)
#         xx=np.append(xx,loc_face)
        
        face_image = eye_rgb[top:bottom, left:right]
        face_image=cv2.resize(face_image,(197,197))

        cv2.imwrite(new_path_name+file_naming, face_image, [cv2.IMWRITE_JPEG_QUALITY,100])
      

    
    
    
    
    
    xx2=xx.reshape(1,68)


    # print(xx2)
    # print(xx2.ndim)

            
    return xx2

In [20]:
How_many_frames = []
def count_org_frames(naming,how_many):
    How_many_frames.append([naming,how_many])

In [21]:
#KNN : d:/VMS/trained_knn_model.clf
model_path="d:/VMS/trained_knn_model.clf"
with open(model_path, 'rb') as f:
            KNN_classifier = pickle.load(f)

In [22]:
# SVM d:/VMS/trained_SVM_model.clf
# model_path="d:/VMS/trained_SVM_model.clf"
# with open(model_path, 'rb') as f:
#             SVM_classifier = pickle.load(f)

In [23]:
def predict_KNN(faces_encodings,X_face_locations, distance_threshold=0.5):
    
    # Use the KNN model to find the best matches for the test face
    faces_encodings=faces_encodings.reshape(1, -1)
    closest_distances = KNN_classifier.kneighbors(faces_encodings, n_neighbors=3)
    are_matches = [closest_distances[0][0][0] <= distance_threshold ]

    # Predict classes and remove classifications that aren't within the threshold
    return KNN_classifier.predict(faces_encodings), are_matches

In [24]:
def predict_SVM(faces_encodings,X_face_locations, distance_threshold=0.6):
    faces_encodings=faces_encodings.reshape(1, -1)
        
    closest_distances = SVM_classifier.predict_proba(faces_encodings)[0]
    are_matches = [closest_distances[0]<= distance_threshold ]



    # Predict classes and remove classifications that aren't within the threshold
    return SVM_classifier.predict(faces_encodings), are_matches

In [25]:
def images_sound_decoding(Folder_path, F_Target_Staff):

    Folder_path_IMG= glob.glob(output_path+"/"+Folder_path+'/*_wc_Image/')
    Folder_path_Sound= glob.glob(output_path+"/"+Folder_path+'/*_wc_Sound/')


    start_time = datetime.now()
    sceneDF0 = pd.DataFrame()
    mp4_Frames=[]
    FrameNames=[]
    face_landmarks=np.array([[]])
    Frames_list=os.listdir(Folder_path_IMG[0])
    

    #sorting the list
    Frames_list = sorted(Frames_list, key = lambda x: int(x.split("_")[-1].split(".")[0]))

    landmark_features=np.array(([[]]),dtype=np.float32)
    #print(Frames_list)
    i=0
    print('encoding {} related to {} Frames, In Progress .. ..'.format(len(Frames_list),F_Target_Staff) )
    count_org_frames(Folder_path,len(Frames_list))
    for clss2 in Frames_list:
        curFrm = face_recognition.load_image_file(f'{Folder_path_IMG[0]}/{clss2}')
        mp4_Frames.append(curFrm)
        FrameNames.append(os.path.splitext(clss2)[0])
        img=curFrm
        lessSize= cv2.resize(img,(224,224))
#         lessSize=cv2.cvtColor(lessSize,cv2.COLOR_BGR2RGB) #conv image to RGB (search about it)
        for_eye= cv2.imread(f'{Folder_path_IMG[0]}/{clss2}',1)
        for_eye=cv2.resize(img,(224,224))

        
        facesInfrm=face_recognition.face_locations(lessSize,model="cnn") #capture all faces in the frame incase many / use (lessSize,model="cnn") for GPU and accuracy
        encodeCurrentFrm =face_recognition.face_encodings(lessSize,facesInfrm)
        print(' {} .. '.format(i),end ='')
        i=i+1
        frm_flag =False
        if len(facesInfrm)==0:
            del_img= extract_Face_LandMarks(lessSize, f'{Folder_path_IMG[0]}/{clss2}', 0,for_eye) 
            print("||",end ='')
            frm_flag =True

        
        attemp_time =1 
        for encodeFace,faceLoc in zip(encodeCurrentFrm,facesInfrm):
            who999,matches = predict_KNN(encodeFace,faceLoc, distance_threshold=0.5)  ## SVM OR KNN
#             faceDis=face_recognition.face_distance(encodeStaffListKnown,encodeFace)
            #print(faceDis)
            
            
            if matches:
                name=who999[0]
                #print(name)
                face_landmarks=extract_Face_LandMarks(lessSize, f'{Folder_path_IMG[0]}/{clss2}', faceLoc,for_eye)
                
                if len(facesInfrm) > 1 and frm_flag is False and F_Target_Staff ==  name:
                    landmark_features=np.append(landmark_features,face_landmarks )# #check the preferred datatype
                    frm_flag = True
                    print("|*|",end ='')
                    break

                elif len(facesInfrm) == 1 and frm_flag is False and F_Target_Staff ==  name :
                        landmark_features=np.append(landmark_features,face_landmarks )# #check the preferred datatype
                        frm_flag=True
                        print("|**|",end ='')
                elif len(facesInfrm) > 1 and frm_flag is False and attemp_time == len(facesInfrm):
                        del_img= extract_Face_LandMarks(lessSize, f'{Folder_path_IMG[0]}/{clss2}', 0,for_eye)
                        frm_flag=True
                        attemp_time=attemp_time+1
                        print("|***|",end ='')
                        
                
                elif len(facesInfrm) == 1 and frm_flag is False and attemp_time == len(facesInfrm):
                    del_img= extract_Face_LandMarks(lessSize, f'{Folder_path_IMG[0]}/{clss2}', 0,for_eye)
                    frm_flag=True
                    attemp_time=attemp_time+1
                    print("|***|",end ='')
                    
                    
                else:
                        attemp_time=attemp_time+1


                
                sceneDF0= SceneFun(name,sceneDF0)
                
            else:
                if frm_flag==False and attemp_time == len(facesInfrm):
                    face_landmarks= extract_Face_LandMarks(lessSize, f'{Folder_path_IMG[0]}/{clss2}', 0,for_eye)
                    frm_flag=True
                    print("|****|",end ='')
                attemp_time=attemp_time+1






    end_time = datetime.now()            
    print('\n Duration: {}'.format(end_time - start_time))

    return sceneDF0,landmark_features,face_landmarks

In [ ]:
mp4_Frames_Path=os.listdir(output_path)
print ("Total of Dir =" + str(len(mp4_Frames_Path)) )
counter=1
for frmspath in mp4_Frames_Path:  #loop Dir to send to function!
    Folder_name=os.path.splitext(frmspath)[0]
    sceneDF999,landmark_features_result,face_landmarks_result=images_sound_decoding(frmspath, staff_Is(frmspath))
    landmark_features_result=landmark_features_result.reshape(-1,68)
    lmf="D:/VMS/del/landmark_features/"+Folder_name+ "_landmark_features_"+ str(counter) +".csv"
    who_was="D:/VMS/del/"+Folder_name+ "_Sample_Validation_"+ str(counter)  +".xlsx"
    np.savetxt(lmf, landmark_features_result, delimiter=",")
    sceneDF999.to_excel(who_was)  
    counter = counter +1



Total of Dir =60
encoding 600 related to 101267 Frames, In Progress .. ..
 0 .. .........|**| 1 .. .........|**| 2 .. .........|**| 3 .. .

E:\Users\sodic\miniconda3\envs\face_recognition\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


........|**| 4 .. .........|**| 5 .. .........|**| 6 .. .........|***| 7 .. .........|***| 8 .. .........|**| 9 .. .........|***| 10 .. .........|***| 11 .. .........|**| 12 .. .........|***| 13 .. .........|***| 14 .. .........|***| 15 .. .........|**| 16 .. .........|***| 17 .. .........|**| 18 .. .........|***| 19 .. .........|***| 20 .. .........|**| 21 .. .........|***| 22 .. .........|***| 23 .. .........|***| 24 .. .........|***| 25 .. .........|***| 26 .. .........|***| 27 .. .........|**| 28 .. .........|**| 29 .. .........|***| 30 .. .........|***| 31 .. .........|**| 32 .. .........|**| 33 .. .........|**| 34 .. .........|**| 35 .. .........|**| 36 .. .........|**| 37 .. .........|**| 38 .. .........|**| 39 .. .........|***| 40 .. .........|**| 41 .. .........|**| 42 .. .........|**| 43 .. .........|**| 44 .. .........|***| 45 .. .........|***| 46 .. .........|**| 47 .. || 48 .. || 49 .. .........|**| 50 .. .........|**| 51 .. || 52 .. || 53 .. || 54 .. .........|***| 55 .. 

E:\Users\sodic\miniconda3\envs\face_recognition\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


........|***| 33 .. .........|**| 34 .. || 35 .. .........|**| 36 .. .........|**| 37 .. .........|**| 38 .. .........|**| 39 .. .........|**| 40 .. .........|**| 41 .. .........|**| 42 .. .........|***| 43 .. .........|***| 44 .. .........|**| 45 .. .........|**| 46 .. .........|**| 47 .. .........|**| 48 .. .........|**| 49 .. .........|**| 50 .. .........|**| 51 .. .........|**| 52 .. .........|**| 53 .. .........|**| 54 .. .........|**| 55 .. .........|**| 56 .. .........|**| 57 .. .........|**| 58 .. .........|**| 59 .. .........|**| 60 .. .........|**| 61 .. .........|**| 62 .. .........|**| 63 .. .........|**| 64 .. .........|**| 65 .. .........|**| 66 .. .........|**| 67 .. .........|**| 68 .. .........|**| 69 .. .........|***| 70 .. .........|***| 71 .. || 72 .. || 73 .. || 74 .. || 75 .. || 76 .. || 77 .. || 78 .. || 79 .. || 80 .. || 81 .. || 82 .. || 83 .. || 84 .. || 85 .. || 86 .. || 87 .. || 88 .. || 89 .. || 90 .. || 91 .. || 92 .. || 93 .. || 94 .. || 95 .. || 96 .. ||

...

.........|***| 12 .. .........|***| 13 .. .........|***| 14 .. || 15 .. .........|***| 16 .. || 17 .. .........|***| 18 .. .........|***| 19 .. .........|***| 20 .. .........|***| 21 .. .........|***| 22 .. .........|***| 23 .. .........|***| 24 .. .........|***| 25 .. .........|***| 26 .. .........|***| 27 .. .........|***| 28 .. .........|***| 29 .. .........|***| 30 .. .........|***| 31 .. .........|***| 32 .. .........|***| 33 .. .........|***| 34 .. .........|***| 35 .. .........|***| 36 .. .........|***| 37 .. .........|***| 38 .. .........|***| 39 .. .........|***| 40 .. .........|***| 41 .. .........|***| 42 .. .........|***| 43 .. .........|***| 44 .. || 45 .. || 46 .. .........|***| 47 .. .........|***| 48 .. .........|***| 49 .. .........|***| 50 .. .........|***| 51 .. .........|***| 52 .. .........|***| 53 .. .........|***| 54 .. .........|***| 55 .. .........|***| 56 .. .........|***| 57 .. .........|***| 58 .. .........|***| 59 .. .........|***| 60 .. .........|***| 61 .

E:\Users\sodic\miniconda3\envs\face_recognition\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


 1 .. .........|***| 2 .. .........|***| 3 .. || 4 .. .........|***| 5 .. .........|***| 6 .. .........|***| 7 .. .........|***| 8 .. .........|***| 9 .. .........|***| 10 .. .........|***| 11 .. .........|***| 12 .. .........|***| 13 .. .........|***| 14 .. .........|***| 15 .. .........|***| 16 .. .........|***| 17 .. .........|***| 18 .. .........|***| 19 .. .........|***| 20 .. .........|***| 21 .. .........|***| 22 .. .........|***| 23 .. .........|***| 24 .. .........|***| 25 .. .........|***| 26 .. .........|***| 27 .. .........|***| 28 .. .........|***| 29 .. .........|***| 30 .. .........|***| 31 .. .........|***| 32 .. .........|***| 33 .. .........|***| 34 .. .........|***| 35 .. .........|***| 36 .. .........|***| 37 .. .........|***| 38 .. .........|***| 39 .. .........|***| 40 .. .........|***| 41 .. .........|***| 42 .. .........|***| 43 .. .........|***| 44 .. .........|***| 45 .. .........|***| 46 .. .........|***| 47 .. .........|***| 48 .. .........|***| 49 .. ......

E:\Users\sodic\miniconda3\envs\face_recognition\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


.........|**| 4 .. .........|**| 5 .. .........|**| 6 .. .........|**| 7 .. .........|**| 8 .. || 9 .. .........|**| 10 .. .........|**| 11 .. .........|**| 12 .. .........|**| 13 .. .........|**| 14 .. || 15 .. .........|**| 16 .. .........|**| 17 .. .........|**| 18 .. .........|***| 19 .. .........|**| 20 .. .........|**| 21 .. .........|**| 22 .. .........|**| 23 .. .........|**| 24 .. .........|**| 25 .. .........|**| 26 .. .........|**| 27 .. .........|**| 28 .. .........|**| 29 .. .........|**| 30 .. .........|**| 31 .. .........|**| 32 .. .........|***| 33 .. || 34 .. || 35 .. .........|**| 36 .. .........|**| 37 .. || 38 .. || 39 .. .........|***| 40 .. .........|**| 41 .. .........|**| 42 .. .........|**| 43 .. .........|**| 44 .. .........|**| 45 .. .........|**| 46 .. .........|**| 47 .. .........|**| 48 .. .........|**| 49 .. .........|**| 50 .. .........|**| 51 .. .........|**| 52 .. || 53 .. || 54 .. .........|**| 55 .. .........|**| 56 .. .........|**| 57 .. .........|*

In [ ]:
How_many_frames=pd.DataFrame(How_many_frames,columns =['Img_name', 'Total_Frames'])
How_many_frames.to_excel("D:/VMS/del/counting.xlsx")

In [ ]:
How_many_frames

In this stage features are ready to Model .